In [1]:
import os
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import nltk
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pickle

In [2]:
table=pd.read_csv(os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\state_temp.csv', low_memory=False)

In [3]:
table['Own_Type'] =np.nan

In [4]:
null = table.loc[pd.isnull(table['Simple_Owners']) == True]
table = table.loc[~pd.isnull(table['Simple_Owners']) == True]

KeyError: 'Simple_Owners'

In [5]:
null['Own_Type'] = 45

In [6]:
table['Simple_Owners'] = table['Simple_Owners'].fillna('NoData')

In [7]:
#initial family 

In [8]:
family = table.loc[(table['initial_class'] == 1) | (table['initial_class'] == 3)]
other = table.loc[(table['initial_class'] != 1) & (table['initial_class'] != 3)]

In [9]:
trust_kw = [' trust ', ' rev tr of ']

In [10]:
trusts = other[other['Simple_Owners'].str.contains('|'.join(trust_kw))]
other = other[~other['Simple_Owners'].str.contains('|'.join(trust_kw))]

In [11]:
searchfor=['revocable', 'living', 'family', ' rev tr of', ' revoc trust', ' revoc ']
family_trusts= trusts[trusts['Simple_Owners'].str.contains('|'.join(searchfor))]
unknown_trusts= trusts[~trusts['Simple_Owners'].str.contains('|'.join(searchfor))]

In [12]:
#hard coding unk trusts
#other=pd.concat([other, unknown_trusts])
unknown_trusts['Own_Type'] = 41

<ipython-input-12-8392cf983357>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unknown_trusts['Own_Type'] = 41


In [13]:
family=pd.concat([family, family_trusts])

In [14]:
#farms

In [15]:
farms= other[other['Simple_Owners'].str.contains(" farms ")]
other= other[~other['Simple_Owners'].str.contains(" farms ")]

In [16]:
searchfor=[' family ', ' brother ', ' son ', ' daughter ']
family_farms= farms[farms['Simple_Owners'].str.contains('|'.join(searchfor))]
unknown_farms= farms[~farms['Simple_Owners'].str.contains('|'.join(searchfor))]

In [17]:
family=pd.concat([family, family_farms])

In [18]:
other=pd.concat([other, unknown_farms])

In [21]:
searchfor=[' family ', ' brother ', ' son ', ' daughter ']

other_family_language = other[other['Simple_Owners'].str.contains('|'.join([x.lower() for x in searchfor]))]
other= other[~other['Simple_Owners'].str.contains('|'.join([x.lower() for x in searchfor]))]

In [22]:
family=pd.concat([family, other_family_language])

In [19]:
#corprate

In [20]:
corp_keywords=[" BANK ", "CORPORATION", "PARTICIPATION", "PROPERTIES",
          "ASSOCIATIONS", "ASSOCIATES", " ASSOCI", "MAINTENANCE", 
               "MAINTENANC", "TELEPHONE","ELECTRIC", "ENTERPRISES", "ENTERPRISE", 
               "AUTHORITY", "HOMEOWNERS", "INTERNATIONAL", " OFFICE", "INVESTMENT", 
               " MGMT"," INN", "ASSOCIATION", "ACCOUNTING", "MAINTEN", "PRODUCT", "MUTUAL", 
         " ESTATES", " & SONS", "FINANCE", " TITLE", "WIRELESS", "COMMUNICATION", 
               " SERVICE", "BAR & GRILL", " DEPT", " CTR", " LOAN", "SPECIALTIES", "BRANDS", " UNITED"
         "CREDIT", " UNION", "CORPORATE", "TREASURER", " ADMIN", "UTILITIES", 
         "COMMERCIAL", " STORAGE", "REAL ESTATE", "CREDIT UNION"]  

biz_word_drop=[" CORP", " LLC", " INC", " LTD", " HRS", "MGT", " OF ", " CO ", " LP", " MGMT"
               " STE ", " L L C", " L P"]

In [23]:
#for component in map(lambda x:x.lower(),biz_word_drop):
      #  other_family_language['Simple_owners']=other_family_language['Simple_owners'].str.replace(component ,'')
#other_family_language_corp = other_family_language[other_family_language['Simple_owners'].str.contains('|'.join(map(lambda x:x.lower(),corp_keywords)))]
#other_family_language_family = other_family_language[~other_family_language['Simple_owners'].str.contains('|'.join(map(lambda x:x.lower(),corp_keywords)))]

In [ ]:
corp = other[other['Simple_Owners'].str.contains('|'.join([x.lower() for x in corp_keywords_keywords]))]
other = other[~other['Simple_Owners'].str.contains('|'.join([x.lower() for x in corp_keywords_keywords]))]

In [ ]:
#government

In [24]:
keywords=[" BANK ", " CORP", "PARTICIPATION", " TRUST", " TRUS", " CO ", " LP"
          "UNIVERSITY", "COLLEGE", " CHURCH"," CLUB",
          "BAPTISI", "EVANGELICAL", "METHODIST", "CATHOLIC", "PROPERTIES",
          "ASSOCIATIONS", "ASSOCIATES"," ASSOCI","SOCIETY", "MAINTENANCE", 
          "MAINTENANC", "ENTERPRISES", "ENTERPRISE", "AUTHORITY", "HOMEOWNERS",
         "INTERNATIONAL", "INVESTMENT", " HOME", " MGMT", " PRESBYTERIAN", " INN", "ASSOCIATION",
          " STE ", "ACCOUNTING", "MAINTEN", "PRODUCT", "MUTUAL", 
         "ESTATES", "PARTNER", " & SONS", "FINANCE", " TITLE", " L P ", " FARM",
         "WIRELESS", "COMMUNICATION", " SERVICE", "BAR & GRILL", " CTR ", " LOAN", "SPECIALTIES", "BRANDS",
         "CREDIT", "CORPORATE", "TREASURER", " ADMIN", "UTILITIES", "COMMERCIAL", " STORAGE", "FAMILY"]    

government_keywords=[' state ', ' county ', ' federal ', ' government ', ' city of ', ' town of ',
                     ' township ', ' department ', ' bureau ']

government = other[other['Simple_Owners'].str.contains('|'.join([x.lower() for x in government_keywords]))]
other = other[~other['Simple_Owners'].str.contains('|'.join([x.lower() for x in government_keywords]))]

In [25]:
gov=government[~government['Simple_Owners'].str.contains('|'.join([x.lower() for x in keywords]))]
unk_gov = government[government['Simple_Owners'].str.contains('|'.join([x.lower() for x in keywords]))]
other = pd.concat([other, unk_gov])

In [26]:
gov['Own_Type'] = 0

<ipython-input-26-5a4aad64b27b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gov['Own_Type'] = 0


In [27]:
#searchfor=[' family ', ' brother ', ' son ', ' daughter ']
#family_government= government[government['Simple_owners'].str.contains('|'.join(searchfor))]
#unknown_government= government[~government['Simple_owners'].str.contains('|'.join(searchfor))]

In [28]:
#religious

In [29]:
rel_key_words= [' Adventist ', ' Baptist ', ' Brethren ', ' Catholic ',
                   'Christian Church', ' Church of ', ' Episcopal ', ' Anglican ',
                   ' Lutheran ', ' Mennonite ', ' Methodist ', ' Moravian ', 'Mormon ', 
                  ' Nazarene ', ' Orthodox ', 'Pentecostal ', ' Presbyterian ', 
                  ' Reformed Church ', 'Spiritualist ', ' Mosques ', ' Islamic Center ',
                  ' Mosque ', ' Masjid ', ' Synagogues ', ' Temple ', ' Congregation '
               , ' of bethlehem ', 'kingdom of god', 'christian*church']


In [30]:
religious_groups = other[other['Simple_Owners'].str.contains('|'.join([x.lower() for x in rel_key_words]))]
other = other[~other['Simple_Owners'].str.contains('|'.join([x.lower() for x in rel_key_words]))]

In [31]:
religious_groups['Own_Type'] = 43

In [32]:
family['Own_Type'] = 45

In [33]:
ckw = ['Rayonier', 'Weyerhaeuser', 'Plum Creek'] 
corporate = other[other['Simple_Owners'].str.contains('|'.join([x.lower() for x in ckw]))]
other = other[~other['Simple_Owners'].str.contains('|'.join([x.lower() for x in ckw]))]

In [35]:
corporate['Own_Type'] = 41

In [36]:
kw42 = ['Nature Conservancy']
c42 = other[other['Simple_Owners'].str.contains('|'.join([x.lower() for x in kw42]))]
other = other[~other['Simple_Owners'].str.contains('|'.join([x.lower() for x in kw42]))]

In [37]:
c42['Own_Type'] = 42

In [38]:
kw43 = ['YMCA', 'YWCA', 'Boys Scouts', 'Girls Scouts']
c43 = other[other['Simple_Owners'].str.contains('|'.join([x.lower() for x in kw43]))]
other = other[~other['Simple_Owners'].str.contains('|'.join([x.lower() for x in kw43]))]

In [39]:
c43['Own_Type'] = 43

In [40]:
other['Simple_Owners'] = other.Simple_Owners.map(lambda x: str(x).lower())
other['Simple_Owners']= other.Simple_Owners.str.replace('[^\w\s]', '')
other['Simple_Owners'] = other['Simple_Owners'].apply(nltk.word_tokenize)

<ipython-input-40-3c01850ad120>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  other['Simple_Owners']= other.Simple_Owners.str.replace('[^\w\s]', '')


In [41]:
stemmer =PorterStemmer()
other['Simple_Owners']=other['Simple_Owners'].apply(lambda x: [stemmer.stem(y) for y in x])

In [42]:
other['Simple_Owners']=other['Simple_Owners'].apply(lambda x: ' '.join(x))

In [43]:
count_vect = CountVectorizer()
counts = count_vect.fit_transform(other['Simple_Owners'])

In [44]:
transformer = TfidfTransformer().fit(counts)

counts=transformer.transform(counts)

In [45]:
count_dict = {k: v for k, v in sorted(count_vect.vocabulary_.items(), key=lambda item: item[1])}

In [46]:
x=pd.DataFrame(counts.todense())

In [47]:
with open(os.path.dirname(os.getcwd())+'\\OUTPUTS\\model_dict.pkl', 'rb') as handle:
    model_dict = pickle.load(handle)

In [48]:
model_dict = {k: v for k, v in sorted(model_dict.items(), key=lambda item: item[1])}

In [49]:
model_df = pd.DataFrame(model_dict, index = [0])

In [50]:
model_df = model_df.drop(0)

In [51]:
from tqdm import tqdm
for i in tqdm(list(model_dict.keys())):
    if i in list(count_dict.keys()):
        model_df[i] = x.iloc[:,count_dict[i]]
    else:
        model_df[i] = pd.Series(np.zeros((len(x))))

100%|█████████████████████████████████████████████████████████████████████████████| 4810/4810 [00:12<00:00, 392.14it/s]


In [52]:
loaded_model=pickle.load(open(os.path.dirname(os.getcwd())+'\\OUTPUTS\\classify_unknown_ownership_model.pkl', 'rb'))

In [53]:
other['Own_Type']=loaded_model.predict(model_df)

In [54]:
total=pd.concat([other, unknown_trusts, family, gov, religious_groups, corporate, c42, c43, null])

In [65]:
table=pd.read_csv(os.path.dirname(os.getcwd())+'\\OUTPUTS\\FINAL\\TABLES\\DE\\DELAWARE_PARCELS_FULL.csv')

In [69]:
table['Simple_Owners'] = table.OWN1.map(lambda x: str(x).lower())
table['Simple_Owners']= table.Simple_Owners.str.replace('[^\w\s]', '')

<ipython-input-69-e8044cf8f04e>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  table['Simple_Owners']= table.Simple_Owners.str.replace('[^\w\s]', '')


In [71]:
table = table.loc[table['OWNCD'] == 0]

In [91]:
searchfor = ['FEDERAL ', 'BUREAU', 'UNITED STATES ']
#searchfor = list(pd.read_csv(os.path.dirname(os.getcwd())+'\\INPUTS\\Federal_Agencies.csv')['Agency'].unique())
#[searchfor.append(i) for i in ['FEDERAL ', 'BUREAU', 'UNITED STATES ']]
#searchfor = [i.capitalize() for i in searchfor]
fed_gov = table[table['OWN1'].str.contains('|'.join(searchfor))]
other_gov = table[~table['OWN1'].str.contains('|'.join(searchfor))]

In [94]:
searchfor=['State ', 'state ', 'STATE ']
state_gov = other_gov[other_gov['OWN1'].str.contains('|'.join(searchfor))]
local_gov = other_gov[~other_gov['OWN1'].str.contains('|'.join(searchfor))]

In [ ]:
fed_gov['OWNCD'] = 'Federal'
state_gov['OWNCD'] = 'State'
local_gov['OWNCD'] = 'Local'

In [56]:
total.to_csv(os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\classified_state_temp.csv')

In [ ]:
print('Ownerships Classified')